# Análise Exploratória de Dados - Absenteísmo

### Data Frame(s):
- Absenteísmo em uma empresa de logística (Brasil) 2007-2010: https://archive.ics.uci.edu/dataset/445/absenteeism+at+work

In [ ]:
# Bibliotecas
from ucimlrepo import fetch_ucirepo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from itertools import combinations

In [ ]:
# Montando DataFrame
absenteeism_at_work = fetch_ucirepo(id=445)
X = absenteeism_at_work.data.features
y = absenteeism_at_work.data.targets
# Junta os features e o target em um único DataFrame
df = pd.concat([X, y], axis=1)
# Visualiza as primeiras linhas
df.head()

In [ ]:
# Estrutura básica do DF
df.info()
df.shape
df.columns
df.dtypes

In [ ]:
# Valores ausentes e duplicados
print("Verificação de dados nulos (soma por coluna):")
print(df.isnull().sum())

print("\nVerificação de dados duplicados (soma):")
print(df.duplicated().sum())

In [ ]:
# Mostrar valores duplicados e seus pares
df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()).head(6)
# Como o objetivo da análise é descobrir o que influencia principalmente no absenteísmo, ocorrências idênticas provavelmente seriam casos legítimos e não erro de inserção de dado

## Detalhamento das colunas do DF
Explicações e contexto adicional para cada variável.

### Individual identification (ID)
- Identificador único de cada funcionário.
- Serve apenas para diferenciar os indivíduos, não contém significado analítico.

### Reason for absence (ICD)
- Código que representa o motivo da ausência (CID).
- 21 categorias principais + 7 extras (22-28).

Exemplos:
I: Doenças infecciosas e parasitárias
II: Neoplasias (tumores)
... (continua para todos os códigos)

### Seasons
- 1: Verão, 2: Outono, 3: Inverno, 4: Primavera
- Ajuda a identificar padrões sazonais de absenteísmo.

### Transportation expense
- Custo mensal de transporte do funcionário.

### Distance from Residence to Work
- Distância entre residência e trabalho em km.

### Service time
- Tempo total de serviço do funcionário na empresa (anos).

### Work load Average/day
- Média de horas produtivas/demanda da empresa.

### Hit target
- Indicador binário de atingimento de metas.

### Disciplinary failure
- Variável binária: 1 = falha disciplinar, 0 = nenhuma.

### Education
- 1: Ensino médio, 2: Graduação, 3: Pós-graduação, 4: Mestrado/Doutorado.

### Social drinker / Social smoker
- 1 = Sim, 0 = Não. Avalia hábitos que podem influenciar saúde e absenteísmo.

### Body mass index
- IMC do funcionário.

### Absenteeism time in hours (target)
- Variável alvo: total de horas de ausência.

## Análise de Correlação entre Variáveis Numéricas

In [ ]:
# Separar variáveis numéricas
var_numericas = [
    "Transportation expense",
    "Distance from Residence to Work",
    "Service time",
    "Work load Average/day ",
    "Hit target",
    "Body mass index",
    "Age",
    "Son",
    "Pet",
    "Weight",
    "Height",
    "Absenteeism time in hours"
]

# Relação entre variáveis numéricas
corr = df[[x for x in var_numericas]].corr(method="pearson")

plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação entre variáveis numéricas')
plt.show()

## Análise de Associação entre Variáveis Categóricas

In [ ]:
var_categoricos = [
    "Reason for absence",
    "Seasons",
    "Disciplinary failure",
    "Education",
    "Social drinker",
    "Social smoker",
    "Month of absence",
    "Day of the week"
]

# Mapas de categorias
reason_for_absence = {
    1: "Doenças infecciosas e parasitárias", 2: "Neoplasias (tumores)", 3: "Doenças do sangue e do sistema imunitário",
    4: "Doenças endócrinas, nutricionais e metabólicas", 5: "Transtornos mentais e comportamentais", 6: "Doenças do sistema nervoso",
    7: "Doenças do olho e anexos", 8: "Doenças do ouvido e do processo mastoide", 9: "Doenças do sistema circulatório",
    10: "Doenças do sistema respiratório", 11: "Doenças do sistema digestivo", 12: "Doenças da pele e tecido subcutâneo",
    13: "Doenças do sistema musculoesquelético e tecido conjuntivo", 14: "Doenças do sistema geniturinário", 15: "Gravidez, parto e puerpério",
    16: "Condições originadas no período perinatal", 17: "Malformações congênitas e anomalias cromossômicas", 18: "Sintomas e achados clínicos anormais não classificados em outro local",
    19: "Lesões, envenenamentos e outras consequências de causas externas", 20: "Causas externas de morbidade e mortalidade", 21: "Fatores que influenciam o estado de saúde e contato com serviços de saúde",
    22: "Acompanhamento de paciente", 23: "Consulta médica", 24: "Doação de sangue", 25: "Exame laboratorial", 26: "Falta não justificada", 27: "Sessão de fisioterapia", 28: "Consulta odontológica"
}
seasons = {1: "Verão", 2: "Outono", 3: "Inverno", 4: "Primavera"}
disciplinary_failure = {1: "Sim", 0: "Não"}
education = {1: "Ensino médio", 2: "Graduado", 3: "Pós-graduado", 4: "Mestre e doutor"}
social_drinker = {1: "Sim", 0: "Não"}
social_smoker = {1: "Sim", 0: "Não"}
month_of_absence = {1:"Janeiro",2:"Fevereiro",3:"Março",4:"Abril",5:"Maio",6:"Junho",7:"Julho",8:"Agosto",9:"Setembro",10:"Outubro",11:"Novembro",12:"Dezembro"}
day_week = {2:"Segunda-feira",3:"Terça-feira",4:"Quarta-feira",5:"Quinta-feira",6:"Sexta-feira"}

# Criar DataFrame categórico
df_cat = df.copy()
df_cat["Reason for absence"] = df_cat["Reason for absence"].map(reason_for_absence)
df_cat["Seasons"] = df_cat["Seasons"].map(seasons)
df_cat["Disciplinary failure"] = df_cat["Disciplinary failure"].map(disciplinary_failure)
df_cat["Education"] = df_cat["Education"].map(education)
df_cat["Social drinker"] = df_cat["Social drinker"].map(social_drinker)
df_cat["Social smoker"] = df_cat["Social smoker"].map(social_smoker)
df_cat["Month of absence"] = df_cat["Month of absence"].map(month_of_absence)
df_cat["Day of the week"] = df_cat["Day of the week"].map(day_week)
df_cat.head()

In [ ]:
# Função Cramér's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Calcular todas combinações
resultados = []
for var1, var2 in combinations(var_categoricos, 2):
    valor = cramers_v(df_cat[var1], df_cat[var2])
    resultados.append({"Variável 1": var1, "Variável 2": var2, "Cramers V": valor})

df_cramers = pd.DataFrame(resultados).sort_values(by="Cramers V", ascending=False).reset_index(drop=True)
print("Correlação entre variáveis categóricas (Cramér’s V):")
display(df_cramers)

In [ ]:
# Boxplots categóricas vs absenteísmo
for i in var_categoricos:
    sns.boxplot(x=i, y='Absenteeism time in hours', data=df)
    plt.title(f"{i} x Absenteeism time in hours")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Gráficos de barras do motivo de ausência
df['Reason'] = df['Reason for absence'].map(reason_for_absence)
df_count = df['Reason'].value_counts().sort_values(ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(x=df_count.index, y=df_count.values)
plt.title('Quantidade de ocorrências por motivo de ausência')
plt.xlabel('Motivo de ausência')
plt.ylabel('Quantidade de registros')
plt.xticks(rotation=45, ha='right')
plt.show()

df_mean_hours = df.groupby('Reason')['Absenteeism time in hours'].mean().sort_values(ascending=False)
plt.figure(figsize=(12,6))
sns.barplot(x=df_mean_hours.index, y=df_mean_hours.values)
plt.title('Média de horas de ausência por motivo')
plt.xlabel('Motivo de ausência')
plt.ylabel('Média de horas')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Análise sobre hábitos não saudáveis
# Álcool
df_mean_drinker = df.groupby('Social drinker')['Absenteeism time in hours'].mean().reset_index()
df_mean_drinker['Drinker status'] = df_mean_drinker['Social drinker'].map({0: 'Não bebe', 1: 'Bebe'})
plt.figure(figsize=(6,5))
sns.barplot(x='Drinker status', y='Absenteeism time in hours', data=df_mean_drinker, palette=['skyblue','salmon'])
plt.title('Média de horas de absenteísmo por hábito de consumo de álcool')
plt.xlabel('Hábito de consumo de álcool')
plt.ylabel('Média de horas de absenteísmo')
plt.ylim(0, df['Absenteeism time in hours'].max() + 1)
plt.show()

# Fumo
df_mean_smoker = df.groupby('Social smoker')['Absenteeism time in hours'].mean().reset_index()
df_mean_smoker['Smoker status'] = df_mean_smoker['Social smoker'].map({0: 'Não fuma', 1: 'Fuma'})
plt.figure(figsize=(6,5))
sns.barplot(x='Smoker status', y='Absenteeism time in hours', data=df_mean_smoker, palette=['skyblue','salmon'])
plt.title('Média de horas de absenteísmo por hábito de fumo')
plt.xlabel('Hábito de fumo')
plt.ylabel('Média de horas de absenteísmo')
plt.ylim(0, df['Absenteeism time in hours'].max() + 1)
plt.show()

# Médias por motivo e hábitos
df['Smoker status'] = df['Social smoker'].map({0: 'Não fuma', 1: 'Fuma'})
df['Drinker status'] = df['Social drinker'].map({0: 'Não bebe', 1: 'Bebe'})

df_smoker = df.groupby(['Reason', 'Smoker status'])['Absenteeism time in hours'].mean().reset_index()
plt.figure(figsize=(14,6))
sns.barplot(x='Reason', y='Absenteeism time in hours', hue='Smoker status', data=df_smoker, palette=['skyblue','salmon'])
plt.title('Média de horas de absenteísmo por motivo de falta (Fumantes)')
plt.xlabel('Motivo de ausência')
plt.ylabel('Média de horas')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Fumante')
plt.show()

df_drinker = df.groupby(['Reason', 'Drinker status'])['Absenteeism time in hours'].mean().reset_index()
plt.figure(figsize=(14,6))
sns.barplot(x='Reason', y='Absenteeism time in hours', hue='Drinker status', data=df_drinker, palette=['skyblue','salmon'])
plt.title('Média de horas de absenteísmo por motivo de falta (Bebedores)')
plt.xlabel('Motivo de ausência')
plt.ylabel('Média de horas')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Bebedor')
plt.show()